# General

## Dataset
https://www.kaggle.com/datasets/blastchar/telco-customer-churn

# TODO:
### David:
- pre-processing
  - converting to numerical data (one hot encoding (pandas), label-encoding/method encoding ?)
  - feature scaling (standardize or normalize ?)
- feature selection
- performance visualization


In [2]:
## Imports & Settings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# display all columns and rows:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [12]:
## Dataset & Preprocessing

# Dataset
df_original = pd.read_csv('dataset/telco-customer-churn.csv', index_col=0)
df = df_original.copy()

# Label Encoding (converting categorical to numerical)
categorical_columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines','InternetService','OnlineSecurity', 'OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod']

# convert and replace categorical columns with numerical data from categorical
for col in categorical_columns:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes

## Model
X = df.drop("Churn",axis=1)
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

            gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
customerID                                                                     
7590-VHVEG       0              0        1           0       1             0   
5575-GNVDE       1              0        0           0      34             1   
3668-QPYBK       1              0        0           0       2             1   
7795-CFOCW       1              0        0           0      45             0   
9237-HQITU       0              0        0           0       2             1   

            MultipleLines  InternetService  OnlineSecurity  OnlineBackup  \
customerID                                                                 
7590-VHVEG              1                0               0             2   
5575-GNVDE              0                0               2             0   
3668-QPYBK              0                0               2             2   
7795-CFOCW              1                0               2 